In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, StandardScaler
import torch
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import csv

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
torch.manual_seed(51)

drive.mount('/gdrive')
%cd /gdrive/My\ Drive/AdultDataset

Mounted at /gdrive
/gdrive/My Drive/AdultDataset


In [ ]:
# Prepare Data:
def read_demographics_and_labels(data_name, number_of_sensitive_attributes=1):
    data = pd.read_csv(data_name)

    print(data.columns)

    # Shuffle Data
    # data = data.sample(frac=1)

    data['y'] = 1
    if data_name == "adult.data":
      data['y'].values[data['income'].values == '<=50K'] = 0
    else:
      data['y'].values[data['income'].values == '<=50K.'] = 0

    if number_of_sensitive_attributes == 2:
      data['s0'] = 0
      data['s0'].values[(data['gender'].values == 'Male') & (data['race'].values == 'White')] = 1

      data['s1'] = 0
      data['s1'].values[(data['gender'].values != 'Male') & (data['race'].values == 'White')] = 1

      data['s2'] = 0
      data['s2'].values[(data['gender'].values == 'Male') & (data['race'].values != 'White')] = 1

      data['s3'] = 0
      data['s3'].values[(data['gender'].values != 'Male') & (data['race'].values != 'White')] = 1

      S = data[['s0', 's1', 's2', 's3']]

    else:
      data['s0'] = 0
      data['s0'].values[(data['gender'].values == 'Male')] = 1

      data['s1'] = 0
      data['s1'].values[(data['gender'].values != 'Male')] = 1

      S = data[['s0', 's1']]

    S_matrix = S.to_numpy()
    return data[['y']].to_numpy(), S_matrix

def read_data(training_data_name, test_data_name):

    training_data = pd.read_csv(training_data_name)

    test_data = pd.read_csv(test_data_name)

    X_train = training_data.to_numpy()
    X_test = test_data.to_numpy()

    X_train = normalize(X_train, axis=0)
    X_test = normalize(X_test, axis=0)
    # sc = StandardScaler()

    # X_train = np.array(X_train)
    # sc.fit(X_train)
    # X_train = sc.transform(X_train)
    # X_test = sc.transform(X_test)

    intercept = X_train.shape[0] * [1]
    intercept_numpy = np.array(intercept)
    intercept_numpy = intercept_numpy[:, np.newaxis]
    X_train = np.append(X_train, intercept_numpy, axis=1)

    intercept = X_test.shape[0] * [1]
    intercept_numpy = np.array(intercept)
    intercept_numpy = intercept_numpy[:, np.newaxis]
    X_test = np.append(X_test, intercept_numpy, axis=1)

    return X_train, X_test


y_train, S_Train = read_demographics_and_labels('adult.data', number_of_sensitive_attributes=2)

y_test, S_Test = read_demographics_and_labels('adult.test', number_of_sensitive_attributes=2)

print(y_train.shape)
print(S_Train.shape)

print(y_test.shape)
print(S_Test.shape)

print(S_Train)
# X_Train, X_Test = read_data('AdultTrain2Sensitive.csv', 'AdultTest2Sensitive.csv')

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(32561, 1)
(32561, 4)
(16281, 1)
(16281, 4)
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]]


In [ ]:
class FERMI(torch.nn.Module):
# class FERMI():

  def __init__(self, X_train, X_test, Y_train, Y_test, S_train, S_test, batch_size=64, epochs=2000, lam=10):

        super(FERMI, self).__init__()

        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
        self.S_train = S_train
        self.S_test = S_test

        self.batch_size = batch_size
        self.epochs = epochs

        self.n = X_train.shape[0]
        self.d = X_train.shape[1]
        self.m = Y_train.shape[1]
        if self.m == 1:
          self.m = 2

        self.k = S_train.shape[1] # 4

        self.W = nn.Parameter(torch.zeros(self.m, self.k)) # k: Support of sensitive attributes, m: number of labels
        self.theta = nn.Parameter(torch.zeros(self.d, 1))

        sums = self.S_train.sum(axis=0) / self.n
        # print(sums)
        # print(sums.shape)
        # print(1/0)
        self.p_s0 = sums[0]
        self.p_s1 = sums[1]
        self.p_s2 = sums[1]
        self.p_s3 = sums[1]

        final_entries = []
        for item in sums:
          final_entries.append(1.0 / np.sqrt(item))

        self.P_s = np.diag(sums)

        self.P_s_sqrt_inv = torch.from_numpy(np.diag(final_entries)).double()
        print(self.P_s_sqrt_inv)
        self.lam = lam


  def forward(self, X):
    outputs = torch.mm(X.double(), self.theta.double())
    logits = torch.sigmoid(outputs)
    return logits


  def grad_loss(self, X, Y):
    outputs = torch.mm(X, self.theta.double())
    probs = torch.sigmoid(outputs)
    return torch.matmul(torch.t(X), probs - Y)

  def fairness_regularizer(self, X, S, f_divergence):

    current_batch_size = X.shape[0]
    summation = 0

    Y_hat = torch.sigmoid(torch.matmul(X, self.theta.double()))
    Y_hat0 = 1 - Y_hat

    p_y1 = torch.mean(Y_hat) # P(y = 1): Taking the average of Y_hat
    p_y0 = 1 - p_y1
    torch.mean(Y_hat)
    # p_s0 = torch.mean(S[:, 0])
    # p_s1 = torch.mean(S[:, 1])
    # p_s2 = torch.mean(S[:, 2])
    # p_s3 = torch.mean(S[:, 3])
    # print(Y_hat.shape)
    # print(S[:, 0])
    # print(S[:, 0].shape)

    p_y1s0 = torch.mean(torch.mul(Y_hat, S[:, 0]))
    p_y1s1 = torch.mean(torch.mul(Y_hat, S[:, 1]))
    p_y1s2 = torch.mean(torch.mul(Y_hat, S[:, 2]))
    p_y1s3 = torch.mean(torch.mul(Y_hat, S[:, 3]))

    p_y0s0 = torch.mean(torch.mul(Y_hat0, S[:, 0]))
    p_y0s1 = torch.mean(torch.mul(Y_hat0, S[:, 1]))
    p_y0s2 = torch.mean(torch.mul(Y_hat0, S[:, 2]))
    p_y0s3 = torch.mean(torch.mul(Y_hat0, S[:, 3]))


    # print(p_y1s0)
    # print(p_y0s1)
    # print(p_y1s1)
    # print(p_y0s0)

    # print(self.W.shape)
    reg = 0
    if f_divergence == 'Chi2':
      term1 = 2 * p_y0s0 * self.W.double()[0][0] - self.p_s0 * p_y0 * self.W.double()[0][0] * self.W.double()[0][0] + self.p_s0 * p_y0 - 2 * p_y0s0
      term2 = 2 * p_y1s0 * self.W.double()[1][0] - self.p_s0 * p_y1 * self.W.double()[1][0] * self.W.double()[1][0] + self.p_s0 * p_y1 - 2 * p_y1s0

      term3 = 2 * p_y0s1 * self.W.double()[0][1] - self.p_s1 * p_y0 * self.W.double()[0][1] * self.W.double()[0][1] + self.p_s1 * p_y0 - 2 * p_y0s1
      term4 = 2 * p_y1s1 * self.W.double()[1][1] - self.p_s1 * p_y1 * self.W.double()[1][1] * self.W.double()[1][1] + self.p_s1 * p_y1 - 2 * p_y1s1

      term5 = 2 * p_y0s2 * self.W.double()[0][2] - self.p_s2 * p_y0 * self.W.double()[0][2] * self.W.double()[0][2] + self.p_s2 * p_y0 - 2 * p_y0s2
      term6 = 2 * p_y1s2 * self.W.double()[1][2] - self.p_s2 * p_y1 * self.W.double()[1][2] * self.W.double()[1][2] + self.p_s2 * p_y1 - 2 * p_y1s2

      term7 = 2 * p_y0s3 * self.W.double()[0][3] - self.p_s3 * p_y0 * self.W.double()[0][3] * self.W.double()[0][3] + self.p_s3 * p_y0 - 2 * p_y0s3
      term8 = 2 * p_y1s3 * self.W.double()[1][3] - self.p_s3 * p_y1 * self.W.double()[1][3] * self.W.double()[1][3] + self.p_s3 * p_y1 - 2 * p_y1s3

      reg = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8

    elif f_divergence == 'KL':
      term1 = p_y0s0 * self.W.double()[0][0] - self.p_s0 * p_y0 * torch.exp(self.W.double()[0][0] - 1)
      term2 = p_y1s0 * self.W.double()[1][0] - self.p_s0 * p_y1 * torch.exp(self.W.double()[1][0] - 1)

      term3 = p_y0s1 * self.W.double()[0][1] - self.p_s1 * p_y0 * torch.exp(self.W.double()[0][1] - 1)
      term4 = p_y1s1 * self.W.double()[1][1] - self.p_s1 * p_y1 * torch.exp(self.W.double()[1][1] - 1)

      term5 = p_y0s2 * self.W.double()[0][2] - self.p_s2 * p_y0 * torch.exp(self.W.double()[0][2] - 1)
      term6 = p_y1s2 * self.W.double()[1][2] - self.p_s2 * p_y1 * torch.exp(self.W.double()[1][2] - 1)

      term7 = p_y0s3 * self.W.double()[0][3] - self.p_s3 * p_y0 * torch.exp(self.W.double()[0][3] - 1)
      term8 = p_y1s3 * self.W.double()[1][3] - self.p_s3 * p_y1 * torch.exp(self.W.double()[1][3] - 1)

      reg = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8
    # print(reg)

    return self.lam * reg

In [ ]:
def fair_training(fermi, batch_size, epochs, initial_epochs = 300, initial_learning_rate = 1, lam=0.1, learning_rate_min = 0.01, learning_rate_max = 0.01, f_divergence='KL'):

  X = fermi.X_train
  S_Matrix = fermi.S_train
  Y = fermi.Y_train
  XTest = fermi.X_test
  STest = fermi.S_test
  YTest = fermi.Y_test

  print(X.shape)
  print(S_Matrix.shape)
  print(Y.shape)

  criterion=torch.nn.BCELoss()

  minimizer = torch.optim.SGD([fermi.theta, fermi.W], lr=initial_learning_rate)
  # maximizer = torch.optim.SGD([fermi.W], lr=learning_rate_max)

  # minimizer_track = []
   # maximizer_track = []

  X_total = torch.from_numpy(X).double()
  Y_total = torch.from_numpy(Y).double()

  for ep in range(epochs + initial_epochs):

      if ep % 100 == 99 or ep > initial_epochs:
        print(ep+1, " epochs:")
        # Test:
        pre_logits = np.dot(XTest, fermi.theta.detach().numpy())
        output_logits = 1/(1 + np.exp(-pre_logits))
        final_preds = output_logits > 0.5
        # print(final_preds.shape)

        # p = 0.3
        # t = p * torch.ones(16281,1)

        # random_numbers = torch.bernoulli(t)
        # print(random_numbers)
        # final_preds = random_numbers * final_preds
        # final_preds = final_preds.numpy()

        test = YTest == 1
        acc = final_preds == test
        true_preds = acc.sum(axis=0)
        print("Accuracy: ", true_preds[0] / output_logits.shape[0] * 100, "%")

        final_preds = np.array(final_preds)
        intersections = np.dot(final_preds.T, STest)
        #print(intersections)
        #print(intersections.shape)
        #print(1/0)
        numbers = STest.sum(axis=0)
        #print(numbers.shape)

        group1 = intersections[0][0] / numbers[0]
        group2 = intersections[0][1] / numbers[1]
        group3 = intersections[0][2] / numbers[2]
        group4 = intersections[0][3] / numbers[3]

        print("DP Violations:")
        print(np.abs(group1 - group2))
        print(np.abs(group1 - group3))
        print(np.abs(group1 - group4))
        print(np.abs(group2 - group3))
        print(np.abs(group2 - group4))
        print(np.abs(group3 - group4))


      number_of_iterations = X.shape[0] // batch_size
      for i in range(number_of_iterations):


          start = i * batch_size
          end = (i+1) * batch_size

          current_batch_X = X[start:end]
          current_batch_Y = Y[start:end]
          current_batch_S = S_Matrix[start:end]

          XTorch = torch.from_numpy(current_batch_X).double()
          logits = fermi(XTorch)
          YTorch = torch.from_numpy(current_batch_Y).double()
          STorch = torch.from_numpy(current_batch_S).double()

          if ep < initial_epochs:
            loss_min = criterion(logits, YTorch)
          else:
            loss_min = criterion(logits, YTorch) + fermi.fairness_regularizer(XTorch, STorch, f_divergence)
          # loss_min = criterion(logits, YTorch)

          minimizer.zero_grad()
          loss_min.backward()

          if ep >= initial_epochs:
            fermi.theta.grad.data.mul_(learning_rate_min / initial_learning_rate) # You can have \eta_w here
            fermi.W.grad.data.mul_(-learning_rate_max / initial_learning_rate) # You can have \eta_w here

          minimizer.step()
  return fermi.theta, fermi.W

In [ ]:
Y_Train, S_Train = read_demographics_and_labels('adult.data', number_of_sensitive_attributes=2)
Y_Test, S_Test = read_demographics_and_labels('adult.test', number_of_sensitive_attributes=2)

print(Y_Train.shape)
print(S_Train.shape)

print(Y_Test.shape)
print(S_Test.shape)

X_Train, X_Test = read_data('AdultTrain2Sensitive.csv', 'AdultTest2Sensitive.csv')

print(X_Train.shape)
print(X_Test.shape)

# fermi_instance = FERMI(X_Train, X_Test, Y_Train, Y_Test, S_Train, S_Test, lam=0.01)

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(32561, 1)
(32561, 4)
(16281, 1)
(16281, 4)
(32561, 60)
(16281, 60)


In [ ]:
# Run FERMI
fermi_instance = FERMI(X_Train, X_Test, Y_Train, Y_Test, S_Train, S_Test, lam=500)
theta_star, W_star = fair_training(fermi_instance, batch_size = 8, epochs=4000, initial_epochs=600, initial_learning_rate=10, learning_rate_min=0.001, learning_rate_max=0.001, f_divergence='KL')

tensor([[1.3031, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.9411, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.5280, 0.0000],
        [0.0000, 0.0000, 0.0000, 3.9108]], dtype=torch.float64)
(32561, 60)
(32561, 4)
(32561, 1)
100  epochs:
Accuracy:  81.50604999692894 %
DP Violations:
0.3016140852525052
0.1247823323823472
0.34695204415944925
0.17683175287015798
0.04533795890694409
0.22216971177710207
200  epochs:
Accuracy:  80.17935016276641 %
DP Violations:
0.32205869602577075
0.13384725717666046
0.3719521552122744
0.18821143884911032
0.04989345918650365
0.23810489803561397
300  epochs:
Accuracy:  79.58970579202752 %
DP Violations:
0.32666730828884016
0.13617290991380737
0.3808034692094937
0.19049439837503282
0.0541361609206535
0.24463055929568633
400  epochs:
Accuracy:  79.24574657576315 %
DP Violations:
0.3323082882438717
0.14338127395430766
0.3854950472459101
0.18892701428956404
0.05318675900203837
0.2421137732916024
500  epochs:
Accuracy:  79.06148270990725 %
DP Violations:
0.3317972

KeyboardInterrupt: ignored